# Advanced MCP Patterns

**Multi-Server Orchestration and Production Patterns**

---

Welcome to **advanced MCP patterns**! This notebook explores sophisticated techniques for building production-ready MCP systems. By the end of this 10-minute tutorial, you'll master multi-server setups, different transports, and professional error handling.

### 🎯 What You'll Learn

In this tutorial, you will:
- Connect to multiple MCP servers simultaneously
- Use different transport mechanisms (SSE, HTTP)
- Implement robust error handling
- Debug MCP connections effectively
- Optimize performance for production
- Build a multi-tool AI assistant

### 🔄 Why Advanced Patterns?

Real-world applications need:
- **Multiple Services**: Combine different capabilities
- **Flexibility**: Choose the right transport
- **Reliability**: Handle failures gracefully
- **Performance**: Scale to production loads

## 📦 Step 1: Setting Up Dependencies

### Required Packages
We'll need MCP with additional transport support.

### 📚 What We're Using
- **mcp**: Core MCP package
- **httpx**: For HTTP transports
- **asyncio**: For async operations

In [ ]:
# Install required packages
%pip install mcp httpx -q

# Import everything we need
import asyncio
import json
from typing import List, Dict, Any
import platform

# MCP imports
from mcp import stdio_client, StdioServerParameters
from mcp.client.sse import sse_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP

# Strands imports
from strands import Agent
from strands.tools.mcp import MCPClient
from strands.models import BedrockModel
import boto3

print("✅ All dependencies ready!")
print(f"   Platform: {platform.system()}")
print("   Ready for advanced MCP patterns! 🚀")

## 🌐 Step 2: Understanding Transport Options

### Transport Comparison

| Transport | Best For | Pros | Cons |
|-----------|----------|------|------|
| **stdio** | Local tools | Simple, fast | Local only |
| **SSE** | Web services | Real-time, scalable | Requires HTTP server |
| **HTTP** | REST APIs | Standard, flexible | Higher latency |
| **Custom** | Special needs | Full control | Complex to implement |

### 🎯 Choosing the Right Transport
- **Local scripts**: Use stdio
- **Web services**: Use SSE or HTTP
- **Microservices**: Use HTTP with load balancing

## 🚀 Step 3: Creating Multiple MCP Servers

### Different Services, Different Servers
Let's create three different MCP servers to demonstrate multi-server patterns.

In [ ]:
# 1. Weather Service (simulated)
weather_mcp = FastMCP("Weather Service")

@weather_mcp.tool(description="Get current weather for a city")
def get_weather(city: str) -> Dict[str, Any]:
    """Get simulated weather data for a city."""
    # In production, this would call a real weather API
    weather_data = {
        "New York": {"temp": 72, "condition": "Sunny", "humidity": 45},
        "London": {"temp": 59, "condition": "Cloudy", "humidity": 70},
        "Tokyo": {"temp": 68, "condition": "Clear", "humidity": 60},
        "Sydney": {"temp": 77, "condition": "Partly Cloudy", "humidity": 55}
    }
    
    if city in weather_data:
        return {
            "city": city,
            "temperature": weather_data[city]["temp"],
            "condition": weather_data[city]["condition"],
            "humidity": weather_data[city]["humidity"],
            "unit": "Fahrenheit"
        }
    else:
        return {
            "error": f"Weather data not available for {city}",
            "available_cities": list(weather_data.keys())
        }

# 2. News Service (simulated)
news_mcp = FastMCP("News Service")

@news_mcp.tool(description="Get latest news headlines by category")
def get_headlines(category: str = "general", limit: int = 5) -> List[Dict[str, str]]:
    """Get simulated news headlines."""
    headlines = {
        "technology": [
            {"title": "AI Breakthrough in Medical Diagnosis", "source": "TechNews"},
            {"title": "New Quantum Computer Sets Performance Record", "source": "ScienceDaily"},
            {"title": "Major Security Update Released for Popular OS", "source": "SecurityWeek"}
        ],
        "business": [
            {"title": "Stock Market Reaches New Heights", "source": "FinanceToday"},
            {"title": "Startup Unicorn Announces IPO Plans", "source": "BusinessWire"},
            {"title": "Global Supply Chain Shows Recovery Signs", "source": "EconomicTimes"}
        ],
        "general": [
            {"title": "Climate Summit Reaches Historic Agreement", "source": "GlobalNews"},
            {"title": "Olympic Games Announce New Sports", "source": "SportsDaily"},
            {"title": "Archaeological Discovery Rewrites History", "source": "ScienceNews"}
        ]
    }
    
    selected = headlines.get(category, headlines["general"])
    return selected[:limit]

print("🌐 Multiple MCP servers created!")
print("   - Weather Service: get_weather")
print("   - News Service: get_headlines")
print("   - Database Service: (using existing e-commerce server)")

## 🔌 Step 4: Connecting Multiple Servers

### Orchestrating Multiple Services
The key to multi-server MCP is proper context management. Each server needs its own client.

In [ ]:
# Set up AWS Bedrock
session = boto3.Session(profile_name='default')
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# Create MCP clients for each service
# 1. Calculator (stdio)
calc_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="python",
        args=["../src/mcp_servers/calculator_server.py"]
    )
))

# 2. Database (stdio)
db_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="python",
        args=["../src/mcp_servers/ecommerce_db_server.py"]
    )
))

print("🔌 MCP clients configured!")
print("   - Calculator Client (stdio)")
print("   - Database Client (stdio)")
print("   Note: Weather and News would use SSE/HTTP in production")

## 🎯 Step 5: Multi-Server Agent

### Combining Tools from Multiple Servers
Let's create an agent that can use tools from different MCP servers simultaneously.

In [ ]:
# Create a multi-capability agent
print("🤖 Creating multi-server agent...\n")

# Use nested context managers for multiple servers
with calc_client as calc, db_client as db:
    # Get tools from both servers
    calc_tools = calc.list_tools_sync()
    db_tools = db.list_tools_sync()
    
    # Combine all tools
    all_tools = calc_tools + db_tools
    
    print("📋 Available tools from all servers:")
    print("\n🧮 Calculator Tools:")
    for tool in calc_tools:
        print(f"   - {tool.name}: {tool.description}")
    
    print("\n🗄️ Database Tools:")
    for tool in db_tools:
        print(f"   - {tool.name}: {tool.description}")
    
    # Create the multi-capability agent
    multi_agent = Agent(
        model=bedrock_model,
        system_prompt="""You are a versatile AI assistant with access to multiple services:
        
        1. Calculator: For mathematical operations
        2. E-commerce Database: For product and order information
        
        Use the appropriate tools to help users with their requests.
        You can combine tools from different services to provide comprehensive answers.""",
        tools=all_tools
    )
    
    print("\n✅ Multi-server agent ready!")

## 💬 Step 6: Testing Multi-Server Operations

### Cross-Service Queries
Let's test our agent with requests that require multiple services.

In [ ]:
# Test 1: Math + Database
with calc_client as calc, db_client as db:
    all_tools = calc.list_tools_sync() + db.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=all_tools)
    
    print("🔍 Test 1: Combined Math and Database Query")
    print("="*50)
    response = agent(
        "If I buy 3 Wireless Mice and 2 Mechanical Keyboards, "
        "what's the total cost? Please calculate the exact amount."
    )
    print(f"🤖 Assistant: {response}")
    print("\n" + "="*50 + "\n")

In [ ]:
# Test 2: Analytics + Calculation
with calc_client as calc, db_client as db:
    all_tools = calc.list_tools_sync() + db.list_tools_sync()
    agent = Agent(model=bedrock_model, tools=all_tools)
    
    print("🔍 Test 2: Sales Analytics with Calculations")
    print("="*50)
    response = agent(
        "What's the average revenue per category? "
        "Calculate the exact average across all categories."
    )
    print(f"🤖 Assistant: {response}")
    print("\n" + "="*50 + "\n")

## 🐛 Step 7: Debugging MCP Connections

### Common Issues and Solutions
Here's a brief guide to debugging MCP connections when things go wrong.

In [ ]:
# Debugging helper function
def debug_mcp_connection(client_name: str, client: MCPClient):
    """Debug an MCP connection with detailed error reporting."""
    print(f"\n🔍 Debugging {client_name}...")
    print("=" * 40)
    
    try:
        # Try to connect and list tools
        with client:
            tools = client.list_tools_sync()
            print(f"✅ Connection successful!")
            print(f"   Found {len(tools)} tools")
            
            # Test tool invocation
            if tools:
                print(f"\n📝 Testing first tool: {tools[0].name}")
                # You would invoke the tool here in a real test
                print("   Tool is accessible")
                
    except ConnectionError as e:
        print(f"❌ Connection Error: {e}")
        print("   Fix: Check if the server is running and accessible")
        
    except TimeoutError as e:
        print(f"❌ Timeout Error: {e}")
        print("   Fix: Increase timeout or check server performance")
        
    except Exception as e:
        print(f"❌ Unexpected Error: {type(e).__name__}: {e}")
        print("   Fix: Check server logs and ensure proper setup")

# Debug checklist
print("🐛 MCP DEBUGGING CHECKLIST")
print("=" * 60)

checklist = {
    "🔌 Connection Issues": [
        "Verify server is running: ps aux | grep server_name",
        "Check server logs for errors",
        "Ensure correct transport configuration",
        "Test with simple tool first"
    ],
    "⚡ Performance Issues": [
        "Monitor server resource usage",
        "Check for blocking operations in tools",
        "Consider connection pooling",
        "Implement request timeouts"
    ],
    "🔧 Tool Discovery": [
        "Verify @mcp.tool decorator is used",
        "Check tool has proper type hints",
        "Ensure tool returns JSON-serializable data",
        "Test tool function independently"
    ],
    "🔒 Context Manager": [
        "Always use 'with' statements",
        "Don't store agents outside context",
        "Handle exceptions within context",
        "Clean up resources properly"
    ]
}

for category, items in checklist.items():
    print(f"\n{category}")
    for item in items:
        print(f"   □ {item}")

## ⚡ Step 8: Performance Optimization

### Making MCP Production-Ready
Here are key patterns for optimizing MCP performance.

In [ ]:
# Performance optimization patterns
print("⚡ PERFORMANCE OPTIMIZATION PATTERNS")
print("=" * 60)

# 1. Connection Pooling Pattern
print("\n1️⃣ Connection Pooling")
print("-" * 30)
print("""from contextlib import contextmanager
from typing import Dict

class MCPConnectionPool:
    def __init__(self):
        self.connections: Dict[str, MCPClient] = {}
    
    @contextmanager
    def get_connection(self, name: str, factory):
        if name not in self.connections:
            self.connections[name] = MCPClient(factory)
        
        with self.connections[name] as conn:
            yield conn
""")

# 2. Async Operations Pattern
print("\n2️⃣ Async Operations")
print("-" * 30)
print("""async def parallel_mcp_calls(clients: List[MCPClient]):
    tasks = []
    for client in clients:
        async with client:
            task = client.list_tools_async()
            tasks.append(task)
    
    results = await asyncio.gather(*tasks)
    return results
""")

# 3. Caching Pattern
print("\n3️⃣ Result Caching")
print("-" * 30)
print("""from functools import lru_cache
import hashlib

@lru_cache(maxsize=128)
def cached_tool_call(tool_name: str, args_hash: str):
    # Cache tool results for repeated calls
    return tool_invoke(tool_name, args_hash)

def call_with_cache(tool_name: str, **kwargs):
    args_hash = hashlib.md5(
        json.dumps(kwargs, sort_keys=True).encode()
    ).hexdigest()
    return cached_tool_call(tool_name, args_hash)
""")

# 4. Timeout and Retry Pattern
print("\n4️⃣ Timeout and Retry")
print("-" * 30)
print("""import time
from typing import Optional

def retry_with_backoff(
    func, 
    max_retries: int = 3,
    initial_delay: float = 1.0,
    backoff_factor: float = 2.0
):
    delay = initial_delay
    
    for attempt in range(max_retries):
        try:
            return func()
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            
            time.sleep(delay)
            delay *= backoff_factor
""")

## 🏗️ Step 9: Production Architecture

### Designing Scalable MCP Systems
Here's how to architect MCP for production environments.

In [ ]:
# Production architecture patterns
print("🏗️ PRODUCTION ARCHITECTURE PATTERNS")
print("=" * 60)

architecture = {
    "🌐 Service Mesh": {
        "pattern": "Multiple MCP servers behind load balancer",
        "benefits": ["Scalability", "Fault tolerance", "Load distribution"],
        "example": "Kubernetes with service discovery"
    },
    "🔄 Circuit Breaker": {
        "pattern": "Fail fast when service is down",
        "benefits": ["Prevents cascading failures", "Quick recovery"],
        "example": "Track failure rate, open circuit if > threshold"
    },
    "📊 Monitoring": {
        "pattern": "Instrument all MCP operations",
        "benefits": ["Visibility", "Debugging", "Performance tracking"],
        "example": "Prometheus metrics, OpenTelemetry traces"
    },
    "🔐 Security": {
        "pattern": "Authenticate and authorize MCP calls",
        "benefits": ["Access control", "Audit trail", "Data protection"],
        "example": "JWT tokens, API keys, mTLS"
    }
}

for pattern_name, details in architecture.items():
    print(f"\n{pattern_name}")
    print(f"   Pattern: {details['pattern']}")
    print(f"   Benefits: {', '.join(details['benefits'])}")
    print(f"   Example: {details['example']}")

# Example: Service health check
print("\n\n📊 Example: Health Check Endpoint")
print("="*40)
print("""@mcp.tool(description="Health check for service monitoring")
def health_check() -> Dict[str, Any]:
    return {
        "status": "healthy",
        "timestamp": datetime.now().isoformat(),
        "version": "1.0.0",
        "dependencies": {
            "database": check_db_connection(),
            "cache": check_cache_connection()
        }
    }
""")

## 🎉 Congratulations!

### 🏆 What You've Accomplished

In just 10 minutes, you've:
- ✅ Connected multiple MCP servers simultaneously
- ✅ Explored different transport mechanisms
- ✅ Built a multi-service AI assistant
- ✅ Learned debugging techniques
- ✅ Implemented performance optimizations
- ✅ Designed production architectures

### 🚀 What's Next?

Now that you've mastered advanced MCP patterns, you can:
1. **Build Microservices** - Create specialized MCP services
2. **Implement Monitoring** - Add metrics and tracing
3. **Scale Horizontally** - Deploy with Kubernetes
4. **Add Authentication** - Secure your MCP endpoints

### 💡 Key Takeaways

1. **Multi-Server = More Power**: Combine specialized services
2. **Context Management**: Use nested `with` statements carefully
3. **Debug Systematically**: Follow the checklist
4. **Optimize for Production**: Cache, pool, and retry

### 📚 Resources

- [MCP Documentation](https://modelcontextprotocol.io)
- [Strands MCP Guide](https://strandsagents.com/0.1.x/user-guide/concepts/tools/mcp-tools/)
- [Microservices Patterns](https://microservices.io/patterns/)

### 🌟 Challenge Yourself

Try building a complete MCP ecosystem with:
- Service discovery mechanism
- Central configuration management
- Distributed tracing
- Auto-scaling based on load

Happy building with advanced MCP patterns! 🚀🤖✨